# Examples of using the spherical_joint library

## Importation of useful libraries

In [1]:
from pyluos import Robot
from spherical_joint import Actuator
import time
import math
import numpy as np

/Users/augustin/.virtualenvs/spherical-joint/lib/python3.6/site-packages/quaternion/numba_wrapper.py:20: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


## Parameters of the actuator
To create the actuator, the Actuator from spherical_joint library and robot from pyluos objects are used.<br>
Then, some parameters are set : <br>
- Encoder resolution
- Initial position of the motors
- Reduction ratio between the motors and the disks
- Size of the disks
- PID parameters (found with the Motor_PID.ipynb notebook)
- Activation of the position controller

In [2]:
a = Actuator()

r = Robot('/dev/cu.usbserial-DN05NM0L')
r.gate.delay=10
r.disk_bottom.rot_position=False
r.disk_middle.rot_position=False
r.disk_top.rot_position=False

###########Setup############

r.disk_bottom.encoder_res = 5
r.disk_middle.encoder_res = 5
r.disk_top.encoder_res = 5


r.disk_bottom.setToZero()
r.disk_middle.setToZero()
r.disk_top.setToZero()


r.disk_bottom.reduction = 232
r.disk_middle.reduction = 232
r.disk_top.reduction = 232


r.disk_bottom.wheel_size = 60.0
r.disk_middle.wheel_size = 60.0
r.disk_top.wheel_size = 60


r.disk_bottom.positionPid = [9,0.02,100]
r.disk_middle.positionPid = [9,0.02,100]
r.disk_top.positionPid = [9,0.02,100]


r.disk_bottom.rot_position_mode(True)
r.disk_middle.rot_position_mode(True)
r.disk_top.rot_position_mode(True)

Connected to "/dev/cu.usbserial-DN05NM0L".
Sending detection signal.
Waiting for route table...
Robot setup.


## Set the motors compliant or not
The motors must not be compliant in order to make them rotate
so this parameter must be False.

In [3]:
r.disk_bottom.compliant = True
r.disk_middle.compliant = True
r.disk_top.compliant = True

In [4]:
r.disk_bottom.compliant = False
r.disk_middle.compliant = False
r.disk_top.compliant = False

## Make the motors go back to zero position
Using this cell allows the motors to go back to their initial position when they have been used.

In [6]:
r.disk_bottom.target_rot_position = 0
r.disk_middle.target_rot_position = 0
r.disk_top.target_rot_position = 0

## Demo 1 : Input a vector and an angle
The platform is oriented according to a vector and rotate on its own Z axis at the same time.

In [7]:
q11,q12,q13 = a.get_angles_from_vector([0,0.7,1],0)
r.disk_top.target_rot_position = q11
r.disk_middle.target_rot_position = q12
r.disk_bottom.target_rot_position = q13

time.sleep(1)
angle = 0
for i in range(720):
    angle=i
    time.sleep(0.01)
    q11,q12,q13 = a.get_angles_from_vector([0,0.7,1],angle)

    r.disk_top.target_rot_position = q11
    r.disk_middle.target_rot_position = q12
    r.disk_bottom.target_rot_position = q13

time.sleep(1)
r.disk_bottom.target_rot_position = 0
r.disk_middle.target_rot_position = 0
r.disk_top.target_rot_position = 0

## Demo 2 : Control the actuator with an IMU
An IMU is stuck on another version of the actuator without motors. The actuator imitates the moves of the IMU

In [ ]:
a.reset_last_angles()
a.reset_offset()
while(True):
    imu_quat = r.Imu_mod.quaternion
    q11,q12,q13 = a.get_angles_from_quaternion(imu_quat[0],imu_quat[1],imu_quat[2],imu_quat[3])
    r.disk_top.target_rot_position = q11
    r.disk_middle.target_rot_position = q12
    r.disk_bottom.target_rot_position = q13

    time.sleep(0.01)

## Demo 3 : Circle drawing
The actuator is drawing a circle trajectory in the air.

In [5]:
Rtest = 4
centerTest = 10
angle=0
t=0
a.reset_last_angles()
a.reset_offset()

while t<2*2*math.pi: #TEST CIRCLE PARAMETRIC EQUATION:
    locus = np.array([Rtest*math.cos(t),Rtest*math.sin(t),centerTest])
    t=t+math.pi/100
    angle = angle
    
    q11,q12,q13 = a.get_angles_from_vector(locus,angle)
    
    r.disk_top.target_rot_position = q11
    r.disk_middle.target_rot_position = q12
    r.disk_bottom.target_rot_position = q13

    
    time.sleep(0.01)

## Demo 4 : Circle drawing and rotation
The actuator draws a circle trajectory in the air and rotates on its own Z axis at the same time.

In [7]:
Rtest = 4
centerTest = 10
angle=0
t=0

a.reset_last_angles()
a.reset_offset()

while t<2*2*math.pi:
    locus = np.array([Rtest*math.cos(t),Rtest*math.sin(t),centerTest])
    t=t+math.pi/1000
    angle = angle+1
    
    q11,q12,q13 = a.get_angles_from_vector(locus,angle)
    
    r.disk_top.target_rot_position = q11
    r.disk_middle.target_rot_position = q12
    r.disk_bottom.target_rot_position = q13


    time.sleep(0.01)

## Demo 5 : Keep the platform horizontal
The plaform's rotations is described by the opposite of the quaternion provided by an IMU stuck on it, so it stays horizontal. Oscillation can be observed.

In [ ]:
a.reset_last_angles()
a.reset_offset()
while(True):
    imu_quat = r.Imu_mod.quaternion
    for i in range (1,4):
        imu_quat[i]=-imu_quat[i]
    q11,q12,q13 = a.get_angles_from_quaternion(imu_quat[0],imu_quat[1],imu_quat[2],imu_quat[3])
    r.disk_top.target_rot_position = q11
    r.disk_middle.target_rot_position = q12
    r.disk_bottom.target_rot_position = q13

    time.sleep(0.02)

## Demo  6 : Keep the platform horizontal without oscillation
The oscillation is deleted but the platform is not perfectly horizontal.

In [ ]:
a.reset_last_angles()
a.reset_offset()
quat_prec = r.Imu_mod.quaternion
for i in range(1,4):
    quat_prec[i]=-quat_prec[i]
quat_final = [0,0,0,0]
while(True):
    imu_quat = r.Imu_mod.quaternion
    for i in range (1,4):
        quat_final[i]=(4*quat_prec[i]-imu_quat[i])/5
    quat_final[0]=math.sqrt(1-(quat_final[1]**2+quat_final[2]**2+quat_final[3]**2))
    quat_prec=quat_final
    q11,q12,q13 = a.get_angles_from_quaternion(quat_final[0],quat_final[1],quat_final[2],quat_final[3])
    r.disk_top.target_rot_position = q11
    r.disk_middle.target_rot_position = q12
    r.disk_bottom.target_rot_position = q13

    time.sleep(0.01)